# Get PASTIS modes

## --- HiCAT --

We will now perform a singular value decomposition (SVD) on the PASTIS matrix to get the PASTIS modes.

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u

import hicat.simulators

## Read the matrix

It's your choice wheter you read the analytical or numerical matrix.

In [ ]:
#matrix = fits.getdata('/Users/ilaginja/Documents/data_from_repos/pastis_data/2019-5-15_001_50nm/matrix_numerical/PASTISmatrix_num_HiCAT_piston_Noll1.fits')
matrix = fits.getdata('/Users/ilaginja/Documents/data_from_repos/pastis_data/2019-4-18_001_100nm/matrix_analytical/PASTISmatrix_piston_Noll1.fits')

plt.figure(figsize=(10, 10))
plt.imshow(matrix)
plt.colorbar()

## Get eigenmodes and eigenvalues

Lucie actually uses an SVD. I need to ask her why she doesn't just get the eigenmodes and eigenvalues directly, since we're dealing with a quadratic, symmetrical matrix.

In [ ]:
evals, evecs = np.linalg.eig(matrix)

In [ ]:
print('evals.shape: {}'.format(evals.shape))
print('evals:\n{}'.format(evals))

In [ ]:
print('evecs.shape: {}'.format(evecs.shape))
#print('evecs:\n{}'.format(evecs))

Which dimension are the eigenvectors in?

We know

$$M \cdot u = \lambda \cdot u$$

so we can test this.

In [ ]:
# Evaluate left and right side of eigenvalue problem for mode number n
n = 17

left = np.dot(matrix, evals[n])
right = np.dot(matrix, evecs[:, n])

# Compare them
print('Are the left and right side the same:')
print(np.allclose(left, right))

### Sort from lowest to highest eigenvalue

In [ ]:
# Sort them
sorted_evals = np.sort(evals)
sorted_indices = np.argsort(evals)
sorted_evecs = evecs[:, sorted_indices]

plt.figure(figsize=(14, 8))
#plt.plot(evals, label='Sorted from eigendecomposition')
plt.plot(sorted_evals, label='Sorted lowest to highest evals')
plt.semilogy()
plt.xlabel('Eigenmodes')
plt.ylabel('Log Eigenvalues')
plt.legend()
#plt.savefig('HiCAT_eigenvals.pdf')

## Mode display for JWST: Poppy HexDM

In [ ]:
import poppy
iris = poppy.dms.HexSegmentedDeformableMirror(name='Iris DM',
                                              rings=2,
                                              flattoflat=15.*u.m,
                                              gap=2*u.cm,
                                              center=False)

In [ ]:
mode = 2   # We start numbering at 0 here, 0-35 (Python nunmbering!)

iris.flatten()
for seg, val in enumerate(sorted_evecs[:, mode]):
    print(val)
    iris.set_actuator(seg+1, val*u.um, 0, 0)

In [ ]:
iris.display(what='opd')

In [ ]:
# Lets compute all modes now
jwst_modes = []
w = poppy.Wavefront(diam=6*15.*u.m, npix=512)

for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    iris.flatten()
    for seg, val in enumerate(sorted_evecs[:, mode]):
        #print(val)
        iris.set_actuator(seg+1, val*u.um, 0, 0)
        
    opd = iris.get_opd(w)
    jwst_modes.append(opd)
        
jwst_modes = np.array(jwst_modes)

In [ ]:
# Plot them all
pmin = -5e-7
pmax = 5e-7

plt.figure(figsize=(12, 24))
for mode in range(int(len(evals)/3)):
    
    plt.subplot(6, 3, mode*3+1)
    plt.imshow(jwst_modes[mode*3], cmap='RdBu', vmin=pmin, vmax=pmax)
    plt.colorbar()
    plt.title('Mode ' + str(mode*3+1))
    
    plt.subplot(6, 3, mode*3+2)
    plt.imshow(jwst_modes[mode*3+1], cmap='RdBu', vmin=pmin, vmax=pmax)
    plt.title('Mode ' + str(mode*3+2))
    plt.colorbar()
    
    plt.subplot(6, 3, mode*3+3)
    plt.imshow(jwst_modes[mode*3+2], cmap='RdBu', vmin=pmin, vmax=pmax)
    plt.title('Mode ' + str(mode*3+3))
    plt.colorbar()
    
#plt.savefig('JWST_modes_piston.pdf')

## Mode display for HiCAT

In [ ]:
hc = hicat.simulators.hicat_sim.HICAT_Sim()
hc.iris_ao = 'iris_ao'
hc.apodizer = 'cnt1_apodizer'
hc.lyot_stop = 'cnt1_apodizer_lyot_stop'
hc.include_fpm = True
print(hc.describe())

Put an eigenmode on the IrisAO DM.

In [ ]:
mode = 0   # We start numbering at 0 here, 0-35 (Python numbering!)

hc.iris_dm.flatten()
for seg, val in enumerate(sorted_evecs[:, mode]):
    #print(val)
    hc.iris_dm.set_actuator(seg+1, val*u.um, 0, 0)

In [ ]:
# Display the mode in all planes
plt.figure(figsize=(14,14))
psf, fields = hc.calc_psf(display=True, return_intermediates=True)

In [ ]:
# Show only the segments
plt.imshow(fields[1].phase)

Now lets get all modes at once!

This version with the HiCAT sim is very slow. Instead, you can just display the modes on the Poppy HexDM below!

In [ ]:
hicat_modes = []
for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    hc.iris_dm.flatten()
    for seg, val in enumerate(sorted_evecs[:, mode]):
        #print(val)
        hc.iris_dm.set_actuator(seg+1, val*u.um, 0, 0)
        
    psf, fields = hc.calc_psf(display=False, return_intermediates=True)
    hicat_modes.append(fields[1].phase)
        
hicat_modes = np.array(hicat_modes)

print('All done.')

In [ ]:
# Plot them all
plt.figure(figsize=(48, 12))
for mode in range(len(evals)):
    
    plt.subplot(12, 3, mode*3+1)
    plt.imshow(hicat_modes[mode].phase)
    plt.title('Mode ' + str(mode+1))
    
#plt.savefig('all_HiCAT_modes.pdf')

### Now do it on a Poppy HexDM simply to make it faster

In [ ]:
import poppy
iris_hc = poppy.dms.HexSegmentedDeformableMirror(name='Iris DM',
                                              rings=3,
                                              flattoflat=15.*u.m,
                                              gap=2*u.cm,
                                              center=False)

In [ ]:
mode = 2   # We start numbering at 0 here, 0-35 (Python nunmbering!)

iris_hc.flatten()
for seg, val in enumerate(sorted_evecs[:, mode]):
    #print(val)
    iris_hc.set_actuator(seg+1, val*u.um, 0, 0)
iris_hc.display(what='opd')

In [ ]:
# Lets compute all modes now for HiCAT with a Poppy Iris DM
hicat_modes = []
w = poppy.Wavefront(diam=8*15.*u.m, npix=1024)

for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    iris_hc.flatten()
    for seg, val in enumerate(sorted_evecs[:, mode]):
        #print(val)
        iris_hc.set_actuator(seg+1, val*u.um, 0, 0)
        
    opd = iris_hc.get_opd(w)
    hicat_modes.append(opd)
        
hicat_modes = np.array(hicat_modes)

In [ ]:
# Plot them all
pmin = -5e-7
pmax = 5e-7

plt.figure(figsize=(12, 48))
for mode in range(int(len(evals)/3)):
    
    plt.subplot(12, 3, mode*3+1)
    plt.imshow(hicat_modes[mode*3], cmap='RdBu', vmin=pmin, vmax=pmax)
    plt.colorbar()
    plt.title('Mode ' + str(mode*3+1))
    
    plt.subplot(12, 3, mode*3+2)
    plt.imshow(hicat_modes[mode*3+1], cmap='RdBu', vmin=pmin, vmax=pmax)
    plt.title('Mode ' + str(mode*3+2))
    plt.colorbar()
    
    plt.subplot(12, 3, mode*3+3)
    plt.imshow(hicat_modes[mode*3+2], cmap='RdBu', vmin=pmin, vmax=pmax)
    plt.title('Mode ' + str(mode*3+3))
    plt.colorbar()
    
#plt.savefig('HiCAT_modes_piston.pdf')

In [ ]:
plt.figure(figsize=(15,15))
iris_hc.display(what='opd')